In [ ]:
from src import MACGenerator, UDP_TX, SlidingBook
import src.TX as TX

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import socket
import struct
import time
import json
import cv2

import sys, os

sys.path.append('..')
import utils.utils as utils
import utils.Auth as Auth


def create_param(X,Y,KEY,DIGESTMOD,PAYLOAD_SIZE_BYTE,QUALITY,ATTACK_PROBABILITY,DURATION,DELAY):
    param = {}
    param['X'] = X.tolist()
    try:
        param['Y'] = Y.tolist()
    except:
        return None
    
    param['KEY'] = KEY
    param['DIGESTMOD'] = DIGESTMOD
    param['PAYLOAD_SIZE_BYTE']= PAYLOAD_SIZE_BYTE
    param['QUALITY'] = QUALITY   # param['QUALITY'] of the JPEG compression
    param['ATTACK_PROBABILITY'] = ATTACK_PROBABILITY  # probability of the attack
    param['DURATION'] = DURATION  # running time of the sender in seconds
    param['DELAY'] = DELAY
    return param


#### test the setup
###### IP, PORRT ######
IP = "10.42.0.15"
PORT = 23422
#######################


X = np.eye(30)
Y = Auth.Get_Y(X)


param = create_param(X = X, Y = Y, KEY="key", DIGESTMOD="SHA256", PAYLOAD_SIZE_BYTE=16, QUALITY=25, ATTACK_PROBABILITY=.95, DURATION=10, DELAY=0)
if param is not None:
    args = {'IP': IP, 'PORT': PORT, 'param': param}
    os.system("python3 tx.py \'"+json.dumps(args)+"\'")

In [ ]:
###### IP, PORRT ######
IP = "10.42.0.15"
PORT = 23422
#######################



################# number of experimnets ####################
qualities = [25]
payload_sizes = [16]
attack_probabilities = [0, 0.01, 0.1, 0.2, 0.3]
digets = ['sha256']
Duration = 30 # Seconds
exp = utils.Load_Experiments("../Examples/Xs.pkl")


total_length_of_experiments = len(qualities)*len(payload_sizes)*len(attack_probabilities)*len(digets)*len(exp)
print("Total runtime for experiments: ", total_length_of_experiments*Duration/60, "minutes")

############################################################

cnt = 0
for i in range(len(exp)):

    X = exp[i]['results']['X']
    Y = utils.Get_Y(X)

    for quality in qualities:
        for payload_size in payload_sizes:
            for attack_prob in attack_probabilities:
                for digest in digets:
                    print("Experiment: ", cnt, " of ", total_length_of_experiments)
                    
                    param = create_param(X = X, Y = Y, KEY="key", DIGESTMOD=digest, PAYLOAD_SIZE_BYTE=payload_size, QUALITY=quality, ATTACK_PROBABILITY=attack_prob, DURATION=Duration, DELAY=0)

                    if param is not None:
                        args = {'IP': IP, 'PORT': PORT, 'param': param}
                        os.system("python3 tx.py \'"+json.dumps(args)+"\'")

                    cnt += 1


